In [16]:
# Fungsi keanggotaan umum
def fungsi_segitiga(x, a, b, c):
    return max(0, min((x - a) / (b - a), (c - x) / (c - b)))

def fungsi_trapesium(x, a, b, c, d):
    """Fungsi keanggotaan bentuk trapesium dengan perlindungan pembagian nol."""
    left = 0
    right = 0
    if b != a:
        left = (x - a) / (b - a)
    else:
        left = 1 if x >= a else 0
    if d != c:
        right = (d - x) / (d - c)
    else:
        right = 1 if x <= d else 0
    return max(0, min(left, 1, right))

In [17]:
# Fuzifikasi untuk setiap variabel input
def fuzzify_age(age_val):
    return {
        # Kategori 'Muda' (< 45 tahun)
        'Muda': fungsi_trapesium(age_val, 0, 0, 35, 45),
        
        # Kategori 'Paruh Baya' (45-60 tahun) - Risiko mulai meningkat
        'ParuhBaya': fungsi_segitiga(age_val, 40, 50, 60),
        
        # Kategori 'Senior' (55-75 tahun) - Risiko meningkat tajam setelah usia 55
        'Tua': fungsi_trapesium(age_val, 55, 65, 70, 75),
        
        # Kategori 'Lansia' (> 75 tahun) - Risiko paling tinggi
        'SangatTua': fungsi_trapesium(age_val, 70, 80, 100, 100)
    }
def fuzzify_glucose(glucose_val):
    return {
        # Kategori 'Normal' (< 100 mg/dL)
        'Normal': fungsi_trapesium(glucose_val, 50, 50, 90, 105),
        
        # Kategori 'Perbatasan' atau Prediabetes (100 - 125 mg/dL)
        'Perbatasan': fungsi_segitiga(glucose_val, 100, 115, 130),
        
        # Kategori 'Tinggi' atau Diabetes (>= 126 mg/dL)
        'Tinggi': fungsi_trapesium(glucose_val, 120, 140, 272, 272)
    }

def fuzzify_bmi(bmi_val):
    return {
        # Kategori 'Kurus' (< 18.5)
        'Kurus': fungsi_trapesium(bmi_val, 10, 10, 17, 18.5),
        
        # Kategori 'Normal' (18.5 - 24.9)
        'Normal': fungsi_trapesium(bmi_val, 17.5, 18.5, 24, 25.5),
        
        # Kategori 'Berlebih' (25.0 - 29.9)
        'Berlebih': fungsi_segitiga(bmi_val, 24.5, 27.5, 31),
        
        # Kategori 'Obesitas' (>= 30.0)
        'Obesitas': fungsi_trapesium(bmi_val, 29, 32, 98, 100)
    }

In [ ]:
def defuzzify_sugeno(w_list, z_list):
    # Cek jika tidak ada aturan yang aktif
    if not w_list or sum(w_list) == 0:
        return 0
    numerator = sum(w * z for w, z in zip(w_list, z_list))
    denominator = sum(w_list)
    
    return numerator / denominator

In [19]:
def apply_rules_sugeno(age_fuzzy, glucose_fuzzy, bmi_fuzzy, hypertension, heart_disease):
    rules = [
        # === KELOMPOK ATURAN: RISIKO SANGAT TINGGI (Skor 85-98) ===
        ('SangatTua', 'Tinggi',     'Obesitas', 98),
        ('Tua',       'Tinggi',     'Obesitas', 95),
        ('SangatTua', 'Tinggi',     'Berlebih', 90),
        ('SangatTua', 'Perbatasan', 'Obesitas', 88),

        # === KELOMPOK ATURAN: RISIKO TINGGI (Skor 65-80) ===
        ('Tua',       'Tinggi',     'Berlebih', 80),
        ('Tua',       'Perbatasan', 'Obesitas', 78),
        ('ParuhBaya', 'Tinggi',     'Obesitas', 75),
        ('SangatTua', 'Normal',     'Normal',   70), 
        ('Tua',       'Tinggi',     'Normal',   68),

        # === KELOMPOK ATURAN: RISIKO SEDANG (Skor 45-65) ===
        ('Tua',       'Normal',     'Berlebih', 65),
        ('ParuhBaya', 'Tinggi',     'Berlebih', 60),
        ('ParuhBaya', 'Perbatasan', 'Obesitas', 58),
        ('Muda',      'Tinggi',     'Obesitas', 55),
        ('Tua',       'Normal',     'Normal',   50),
        
        # === KELOMPOK ATURAN: RISIKO RENDAH (Skor 25-45) ===
        ('ParuhBaya', 'Perbatasan', 'Berlebih', 45),
        ('Muda',      'Tinggi',     'Berlebih', 40),
        ('ParuhBaya', 'Normal',     'Berlebih', 35),
        ('Muda',      'Perbatasan', 'Obesitas', 38),
        ('ParuhBaya', 'Normal',     'Normal',   30),

        # === KELOMPOK ATURAN: RISIKO SANGAT RENDAH (Skor 10-25) ===
        ('Muda',      'Perbatasan', 'Berlebih', 25),
        ('Muda',      'Normal',     'Berlebih', 20),
        ('Muda',      'Normal',     'Normal',   15),
        ('Muda',      'Normal',     'Kurus',    10),
    ]
    w_list = []
    z_list = []
    for age_label, glucose_label, bmi_label, z_base in rules:
        if age_label not in age_fuzzy or glucose_label not in glucose_fuzzy or bmi_label not in bmi_fuzzy:
            continue

        w = min(age_fuzzy[age_label], glucose_fuzzy[glucose_label], bmi_fuzzy[bmi_label])

        if w > 0:
            # Modifikasi Skor Risiko berdasarkan Hipertensi dan Penyakit Jantung
            z_final = z_base
            if hypertension == 1:
                z_final += 15 # Tambah 15 poin jika ada hipertensi
            if heart_disease == 1:
                z_final += 20 # Tambah 20 poin jika ada penyakit jantung
            
            w_list.append(w)
            z_list.append(min(100, z_final)) # Batasi skor maksimal di 100

    return w_list, z_list

In [20]:
def predict_stroke_risk_sugeno(input_data):
    """
    Fungsi utama untuk menjalankan seluruh proses Fuzzy Sugeno.
    `input_data` adalah dictionary, contoh: {'age': 67, 'avg_glucose_level': 228, 'bmi': 36}
    """
    # Fuzifikasi: Mengubah input crisp menjadi derajat keanggotaan fuzzy
    age_fuzzy = fuzzify_age(input_data['age'])
    glucose_fuzzy = fuzzify_glucose(input_data['avg_glucose_level'])
    bmi_fuzzy = fuzzify_bmi(input_data['bmi'])

    # Penerapan Aturan Fuzzy Sugeno
    w_values, z_values = apply_rules_sugeno(
        age_fuzzy, 
        glucose_fuzzy, 
        bmi_fuzzy,
        input_data['hypertension'],
        input_data['heart_disease']
    )
    
    # Defuzifikasi
    final_risk_score = defuzzify_sugeno(w_values, z_values)
    
    return final_risk_score

# Uji Evaluasi Model Mamdani

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# Load dataset
df = pd.read_csv('stroke_preprocessed_for_fuzzy.csv')

# Prediksi skor risiko Sugeno untuk seluruh data
def predict_row(row):
    input_data = {
        'age': row['age'],
        'avg_glucose_level': row['avg_glucose_level'],
        'bmi': row['bmi'],
        'hypertension': row['hypertension'],
        'heart_disease': row['heart_disease']
    }
    return predict_stroke_risk_sugeno(input_data)
df['sugeno_risk_score'] = df.apply(predict_row, axis=1)

thresholds = np.arange(1, 100, 1)
f1_scores = []

# Mengambil label target dari dataset sebenarnya dan hasil prediksi
y_true = df['stroke']

# Mencari threshold optimal berdasarkan F1-Score
thresholds = np.arange(1, 100, 1) 
f1_scores = []
for thresh in thresholds:
    y_pred = (df['sugeno_risk_score'] >= thresh).astype(int)
    f1 = f1_score(y_true, y_pred)
    f1_scores.append(f1)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

# Menggunakan threshold optimal untuk menghasilkan prediksi akhir
y_pred_optimal = (df['sugeno_risk_score'] >= optimal_threshold).astype(int)

acc = accuracy_score(y_true, y_pred_optimal)
f1 = f1_score(y_true, y_pred_optimal)
precision = precision_score(y_true, y_pred_optimal)
recall = recall_score(y_true, y_pred_optimal)
print("=== Evaluasi Fuzzy Sugeno ===")
print(f"Akurasi  : {acc:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"Precision Sugeno: {precision:.3f}")
print(f"Recall Sugeno: {recall:.3f}")




Threshold Optimal ditemukan: 82 (dengan F1-Score tertinggi)
=== Evaluasi Fuzzy Sugeno ===
Akurasi  : 0.8790
F1-score : 0.2426
Precision Sugeno: 0.175
Recall Sugeno: 0.398
